In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

In [4]:
import pandas as pd
import numpy as np

# Generate random revenue data for 3 clients over 45 months
num_clients = 3
num_months = 45
months = pd.date_range(start='2018-01', periods=num_months, freq='M')

# Introduce anomalies by increasing revenue by a significant factor
def introduce_anomalies(revenue_data, anomaly_factor=2):
    indices = np.random.choice(len(revenue_data), size=int(len(revenue_data) * 0.1), replace=False)
    for idx in indices:
        revenue_data[idx] *= anomaly_factor
    return revenue_data

client_data = {f"client_{i+1}": pd.DataFrame({'Month': months, 'Revenue': introduce_anomalies(np.random.uniform(800 + i * 500, 1500 + i * 500, num_months))}) for i in range(num_clients)}


In [2]:
def analyze_anomalies_ensemble(client_name, client_df):
    # Standardize the revenue data
    scaler = StandardScaler()
    revenue_scaled = scaler.fit_transform(client_df[['Revenue']])

    # Fit the models
    isolation_forest = IsolationForest(contamination=0.1).fit(revenue_scaled)
    local_outlier_factor = LocalOutlierFactor(n_neighbors=20, contamination=0.1).fit(revenue_scaled)
    one_class_svm = OneClassSVM(nu=0.1, kernel='rbf', gamma=0.1).fit(revenue_scaled)

    # Predict anomalies
    client_df['IF_Anomaly'] = isolation_forest.predict(revenue_scaled)
    client_df['LOF_Anomaly'] = local_outlier_factor.fit_predict(revenue_scaled)
    client_df['OCSVM_Anomaly'] = one_class_svm.predict(revenue_scaled)

    # Compute anomaly scores
    client_df['AnomalyScore'] = (client_df['IF_Anomaly'] == -1).astype(int) + \
                                (client_df['LOF_Anomaly'] == -1).astype(int) + \
                                (client_df['OCSVM_Anomaly'] == -1).astype(int)

    # Identify anomalies
    anomaly_threshold = 2  # Change this value to adjust the sensitivity
    anomalies = client_df[client_df['AnomalyScore'] >= anomaly_threshold]

    # Print anomalies
    print(f"Anomalies for {client_name}:")
    print(anomalies)
    print("\n")

In [5]:
for client_name, client_df in client_data.items():
    analyze_anomalies_ensemble(client_name, client_df)

Anomalies for client_1:
        Month      Revenue  IF_Anomaly  LOF_Anomaly  OCSVM_Anomaly  \
5  2018-06-30  1763.049468          -1           -1              1   
6  2018-07-31  2114.567825          -1           -1             -1   
9  2018-10-31  2967.254418          -1           -1             -1   
19 2019-08-31   805.974799          -1            1             -1   
39 2021-04-30  2382.831599          -1           -1             -1   

    AnomalyScore  
5              2  
6              3  
9              3  
19             2  
39             3  


Anomalies for client_2:
        Month      Revenue  IF_Anomaly  LOF_Anomaly  OCSVM_Anomaly  \
1  2018-02-28  1381.162799          -1           -1             -1   
3  2018-04-30  3718.532821          -1           -1             -1   
24 2020-01-31  3580.732353          -1           -1             -1   
38 2021-03-31  3706.271939          -1           -1             -1   
44 2021-09-30  2705.840442          -1           -1              

we used an ensemble of three techniques: Isolation Forest, Local Outlier Factor, and One-Class SVM. We first standardized the revenue data, fit the models, and predicted the anomalies. We then computed an anomaly score based on the output of each technique. Anomalies were identified based on an anomaly threshold on this score.

You can adjust the `anomaly_threshold` parameter in the `analyze_anomalies_ensemble` function to control the sensitivity of the ensemble method. A higher value will make it more conservative, requiring more techniques to agree on an anomaly, while a lower value will make it more sensitive, requiring fewer techniques to agree.

This ensemble approach allows you to leverage the strengths of different techniques and can potentially provide more accurate and robust anomaly detection. Keep in mind that you can replace the techniques used in this example with any other techniques that you find suitable for your dataset and use case.
user: How should I get the explanation? Detecting anomaly is one think but I know the answer why is that an anomaly? Good explanation for it with data points 
assistant: Explaining the reason behind an anomaly detection can be challenging, especially for unsupervised machine learning methods. For some techniques, the explanation might not be easily interpretable. However, you can provide some insights based on the characteristics of the data points and the algorithm used. 

Here's a general approach to provide an explanation for the detected anomalies:

1. Visualize the data and detected anomalies: Visualizing the data can help you and others understand the context in which an anomaly occurs. You can create time series plots, box plots, or histograms to visualize the data distribution and highlight detected anomalies.

2. Investigate feature importance or contributions (if applicable): For some algorithms, like tree-based methods, you can investigate the importance or contribution of each feature to the anomaly detection. By understanding which features play a significant role in identifying an anomaly, you can better explain why a particular data point is considered anomalous.

3. Calculate descriptive statistics: You can calculate descriptive statistics like mean, median, standard deviation, skewness, and kurtosis for the overall dataset and subsets of the data (e.g., normal and anomalous instances). Comparing these statistics can provide insights into the differences between normal and anomalous instances.

4. Use Local Interpretable Model-agnostic Explanations (LIME) or SHapley Additive exPlanations (SHAP) for model-agnostic explanations: LIME and SHAP are techniques that can provide interpretable explanations for the output of any machine learning model. By applying these methods to your anomaly detection model, you can understand which features contribute to the model's decision and why a specific data point is considered an anomaly.

5. Analyze the context: Investigate the business context, external factors, or events that might have caused the anomaly. This could include seasonality, changes in customer behavior, or macroeconomic factors.

For example, suppose you detected an anomaly in a client's revenue using the Isolation Forest method. You could provide the following explanations:

1. Visualize the data: Create a time series plot of the client's revenue, highlighting the anomalous data points.
2. Descriptive statistics: Calculate the mean and standard deviation of the revenue and compare them to the anomalous data points to show how much they deviate from the norm.
3. Analyze the context: Investigate if there were any events or external factors that might have caused the anomaly, such as a marketing campaign, a change in the market, or a new competitor.

By following this approach, you can provide a better understanding of the reasons behind the detected anomalies and help decision-makers take appropriate actions. Keep in mind that the explanation may not always be straightforward or easily interpretable, but providing some insights can still be valuable.
